## Reasoning and Acting (ReAct)​

Agent to implement ReAct logic

In [1]:
%load_ext dotenv
%dotenv

In [2]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import AzureChatOpenAI

In [3]:
%pip install pypdf
%pip install pycryptodome
%pip install pymupdf


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("data/Underwriting Guidelines 319E 1.pdf")

data = loader.load()

data[0]

Document(page_content='BMO Insurance\nEffective Date:\nFebruary 26, 2024\n319E (2024/02/26)\nUnderwriting \nGuidelines\n', metadata={'source': 'data/Underwriting Guidelines 319E 1.pdf', 'file_path': 'data/Underwriting Guidelines 319E 1.pdf', 'page': 0, 'total_pages': 27, 'format': 'PDF 1.7', 'title': 'Underwriting Guidelines', 'author': 'BMO Insurance', 'subject': '', 'keywords': '', 'creator': 'Adobe InDesign 19.1 (Macintosh)', 'producer': 'Adobe PDF Library 17.0', 'creationDate': "D:20240126101951-05'00'", 'modDate': "D:20240126111742-05'00'", 'trapped': '', 'encryption': 'Standard V5 R6 256-bit AES'})

In [5]:
from langchain_community.vectorstores import FAISS
from langchain_openai import AzureOpenAIEmbeddings
from langchain.tools.retriever import create_retriever_tool

embeddings = AzureOpenAIEmbeddings(azure_deployment="text-embedding-ada-002", openai_api_version="2023-05-15")
vector = FAISS.from_documents(data, embeddings)
retriever = vector.as_retriever()

retriever_tool = create_retriever_tool(
    retriever,
    "bmo_search",
    "Search for information about insurance underwriting. For any questions about insurance, you must use this tool!",
)

In [6]:
tools = [retriever_tool, TavilySearchResults(max_results=1)]

In [7]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react")

In [8]:
# Choose the LLM to use
llm = AzureChatOpenAI(
    openai_api_version="2024-06-01",
    azure_deployment="gpt-4o",
)

# Construct the ReAct agent
agent = create_react_agent(llm, tools, prompt)

In [9]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [10]:
# agent_executor.invoke({"input": "what is LangChain?"})

In [11]:
agent_executor.invoke(
    {"input": "What are the requirements for an Male applicant aged  70 for $9M ?"}
)



> Entering new AgentExecutor chain...


Error in LangChainTracer.on_tool_end callback: TracerException("Found chain run at ID 582db120-a559-473c-94cf-fcbf7b4bc7d8, but expected {'tool'} run.")


To answer this question, I need to gather information about insurance underwriting requirements for a 70-year-old male applicant seeking $9 million in coverage.

Action: bmo_search
Action Input: insurance underwriting requirements for 70-year-old male seeking $9 million coverageUNDERWRITING REQUIREMENTS – TERM 10, 15, 20, 25, 30
(Effective date: February 26, 2024)
(Including Term riders. For term riders the total face amount should be used in determining the requirements.)
An illustration is required for all Universal Life and Whole Life applications.
Current Amount
Issue Age (Age Nearest)
18-40
41-45
46-50
51-55
56-60
61-70
71-751
$100,000 – $249,999
T
T
T
T
T
TV2
TV3
$250,000 – $300,000
T
T
T
T
T
TV2 
TV3
$300,001 – $500,000
T
T
T
T
T
TV2
TV3
$500,001 – $750,000
T
T
T
T
T
TV3
TV3
$750,001 – $1,000,000
T
T
T
TV2
TV2
TV3
TV3
$1,000,001 – $2,000,000
T
T
T
TV2
TV3
TV3
TV3
$2,000,001 – $3,000,000
T
T
T + A
TV2
TV3
TV3
TV3
$3,000,001 – $5,000,000
T
TV2
TV2
TV3
TV3
TV3
TV3
$5,000,001 – $10,

{'input': 'What are the requirements for an Male applicant aged  70 for $9M ?',
 'output': 'For a 70-year-old male applying for $9 million in coverage, the requirements typically include a Tele-Interview with Vitals, financial third-party verification, and a Motor Vehicle Report.'}